# Examples
**Note**: The examples are found in the following jupyter notebook `xmen/examples/examples.ipynb` where they can be run interactively. Before running the notebook make sure that `xmen` is on your `PYTHONPATH` following the steps given in Setup.

## Defining an experiment
New experiments are defined by specifying **parameters** (as public class attributes) and **execution** (by overloading the `run` method). For example

In [15]:
from xmen.experiment import Experiment
import os

class AnExperiment(Experiment):
    """The doc string of AnExperiment class"""
    
    def __init__(self):
        """The doc string of __init__ experiment class."""
        super(AnExperiment, self).__init__()
        self.a: str = 'h'     # A parameter
        self.b: int = 17      # Another parameter
        
        # Private attributes will not be treated as parameters
        self._c = None
    
    def run(self):
        print(f'The experiment state inside run is {self.status} for experiment {self.name}, a = {self.a}, b = {self.b}')
        
        # Add a message to the self.leave message attribute
        self.leave_message({'Hi': 'I am running!'})
        
        # You are encouraged to write out any files to {self.root}/{self.name}
        # during execution these fields will be populated
        with open(os.path.join(self.directory, 'logs.txt'), 'w') as f:
            f.write('This was written from a running experiment')

There are a few things worth pointing out here:

* Parameters of the class are defined as public attributes. You are encouraged to define parameters as above sepcifying their name, type, default and help all in a single line of code. The `Experiment` class has a metaclass `TypedMeta` (along with any class inheriting from `Experiment`). `TypedMeta` automatically adds any attributes to the doc sting of the `__init__` and the main class body.
* Any attributes begining with `_` are considered pivate and will not be treated as a parameters. These should be used to store dynamic state during experiment execution.
* All experiments inheret several important properties which can be accessed but not set:
    * `root` and `name` together define an experiment folder that is assigned to the experiment. The `directory` property gives `{root}/{name}` for convenience. You may assume that these fields will be populated at the time of execution and are encouraged to use this folder to write out anything that needs to be logged during execution (logs, checkpoints etc.)
    * `status` is one of `'default'`, `'registered`', `'running'`, `'finished'` or `'error'` (more on this to come...)
    * `created` gives the date any parameter was last updated
    * `version` is a dict used to reproduce an experiment exactly
    * `messages` is a dict used to store messages logged during execution 
  These parameters should not be updated by the user directly!

## Experiment specialisation

When initialised an experiment is given a `'default'` status. Experiment with default status:

* **Can** be specialised either by directly accessing the experiments parameters or using the `update()` method 
* **Can** be loaded and saved to a `defaults.yml` file using the `from_yml()` and `to_default()` method calls. This allows experiment config files to be generated quickly and safely from within python
* **Cannot** be run! In order to run they must first be linked with an experiment folder and their state saved and fixed.


In [16]:
# Initialiase a new experiment
exp = AnExperiment()

# Update its parameters
exp.update({'a': 'g'})
exp.b = 19
print(exp)

status: default
created: 11:58AM September 20, 2019
messages:
parameters:
  a: g   # A parameter (default='h')
  b: 19   # Another parameter (default=17)


Here you can see that the parameter `a` has been updated to `'g'`. Its default value is still preserved in the docstring. You can also see the `status` of the experiment is currently `'default'` and that `created` has been updated!

Now lets create a new `defaults.yml` file that can be used to specialise other experiments:

In [17]:
# Save the experiment to '/tmp/defaults.yml'
exp.to_defaults('/tmp')

In [18]:
# Load the defaults.yml into a new experiment
exp2 = AnExperiment()
exp2.from_yml('/tmp/defaults.yml')
print(exp2)

status: default
created: 11:58AM September 20, 2019
messages:
version:
  module: /Users/robweston/xmen/examples/<ipython-input-15-64aecc4b7aea>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 25e54f1a04e7f0ee377715c52235f8a204734325
    branch: with_privates
parameters:
  a: g   # A parameter (default='h')
  b: 19   # Another parameter (default=17)


Notice how the experiment still has `"default"` status and has been loaded with the `a="g"`. The `defaults.yml` file looks like this:

In [19]:
%%bash
# The defaults.yml file looks like this:
echo "$(cat /tmp/defaults.yml)"

_created: 11:58AM September 20, 2019  #  The date the last time the parameters of the model were updated.
_version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/<ipython-input-15-64aecc4b7aea>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 25e54f1a04e7f0ee377715c52235f8a204734325
    branch: with_privates
a: g #  A parameter (default='h')
b: 19 #  Another parameter (default=17)


Experiments can not be run. This will lead to a `ValueError`

In [20]:
# Registered experiments cannot be run:
try:
    exp()
except ValueError as e:
    print('ValueError:', e)

ValueError: An experiment in default status must be registered before it can be executed


## Executing an experiment
In order to be run an experiment object must first be registered. During registration:

* An experiment is assigened an experiment folder `{root}/{name}`. Not to experiments may share the same experiment folder
* The paramters of an experiment become fixed and are preserved in a `params.yml` within the experiement folder.

An experiment can be registered in one of two ways:

1. The `register()` method call
2. Loading a predefined `params.yml` file using the `from_yml()` method call

Once registered an experiment can then be run by calling it. The `__call__` method first updates the status of the experiment to `'running'` (both in its internal state and the its `params.yml`) file before calling the `run()` method. If `run` leaves succesfully the state is updated to `finished` else `error`. Only experiments with `registered` status can be executed 

In [21]:
!rm -f /tmp/a/params.yml
exp.register(root='/tmp', name='a', purpose='An example')

Looking inside the project repository we now have:

In [22]:
%%bash
echo 'Folder now contains:'
ls /tmp/a

Folder now contains:
logs.txt
params.yml


The `params.yml` file contains the following:

In [23]:
!echo "$(cat /tmp/a/params.yml)$"

_root: /tmp  #  The root directory to which the experiment belongs (should not be set)
_name: a #  The name of the experiment (should not be set)
_status: registered #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
_created: 11:58AM September 20, 2019 #  The date the last time the parameters of the model were updated.
_purpose: An example #  The purpose for the experiment (should not be set)
_messages: {} #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
_version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/<ipython-input-15-64aecc4b7aea>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 25e54f1a04e7f0ee377715c52235f8a204734325
    branch: with_privates
a: g #  A parameter (default='h')
b: 19 #  Another parameter (default=17)$


Once registered an experiemnts parameters cannot be updated:

In [24]:
try:
    exp.a = 'something'
except AttributeError as e:
    print('AttributeError:', e)
# print(exp)

An experiment **cannot** be registered to a repository already containing a `params.yml` file:

In [25]:
exp2 = AnExperiment()
try:
    exp2.register(root='/tmp', name='a')
except ValueError as e:
    print('ValueError: ', e)

ValueError:  Experiment folder /tmp/a already contains a params.yml file. An Exeperiment cannot be created in an already existing experiment folder


An experiment object can be loaded from already existing `params.yml` using the method call `from_yaml`:

In [26]:
# print(exp2, '\n')
exp2.from_yml('/tmp/a/params.yml')
print(exp2)

root: /tmp
name: a
status: registered
created: 11:58AM September 20, 2019
purpose: An example
messages:
version:
  module: /Users/robweston/xmen/examples/<ipython-input-15-64aecc4b7aea>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 25e54f1a04e7f0ee377715c52235f8a204734325
    branch: with_privates
parameters:
  a: g   # A parameter (default='h')
  b: 19   # Another parameter (default=17)


Now the parameter `a` has been updated to `g` (from its default `h`). When loading from an already created `params.yml` file the status of the experiment is updated to `registered`. The `created` attribute stores the time that the experiment was registered and root and name have also been filled.

> At the moment (as above) two experiments can share the same `params.yml` file. This may however be undesirable. In the future it may also be worth adding a locked attribute to the params.yml file, which is True when a params.yml file is linked to an Experiment object and False when it has been generated by the `ExperimentManager`

After being registered an experiment may now be run:

In [27]:
print('Before running:')
print(exp._status)

# Run the experiment using the __call__ method
exp()

print('\nAfter running:')
print(exp._status)

Before running:
registered
The experiment state inside run is running for experiment a, a = something, b = 19

After running:
finished


In [28]:
%%bash
echo 'The logs.txt file has been written during the call'
ls '/tmp/a'
echo ''
echo "$(cat /tmp/a/logs.txt)"

The logs.txt file has been written during the call
logs.txt
params.yml

This was written from a running experiment


## Versioning Experiments

Experiments evolve, parameter defaults are altered, new parameters are added, and old parameters deppreciated. Config files soon become outdated and matching config files to experiment versions becomes tricky.

As parameters are part of the class definition they (including their defaults and documentation) evolve with the execution code. If the experiment is defined within a git repository then the `Experiment` class automatically takes care of versioning config files. This allows `defaults.yml` which are no longer compatibile with the current experiment definition to be easily identify. Likewise experiments can be perfectly recreated from the information within a `params.yml` by first checking out the correct experiment version.

The version information is stored in the `version` property including:
* The `'module'` the experiment was defined in
* The name of the experiment `'class'` within the `'module'`
* Available `git` information:
    * The `'local'` git repository the experiement belongs to
    * The `'commit'` hash
    * The `'branch'` it was defined in
    * The `'remote'` repository url

If we now inspect the `params.yml` and `defaults.yml` file created earlier from the `AnExperiment` class we can see that the module and class fields have been populated along with the git information and stored in the version attribute:

In [30]:
%%bash
echo "The params.yml file:"
echo "----------------------"
echo "$(cat /tmp/a/params.yml)"
echo ""
echo "The defaults.yml file:"
echo "----------------------"
echo "$(cat /tmp/defaults.yml)"

The params.yml file:
----------------------
_root: /tmp  #  The root directory to which the experiment belongs (should not be set)
_name: a #  The name of the experiment (should not be set)
_status: finished #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
_created: 11:58AM September 20, 2019 #  The date the last time the parameters of the model were updated.
_purpose: An example #  The purpose for the experiment (should not be set)
_messages: #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
  Hi: I am running!
_version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/<ipython-input-15-64aecc4b7aea>
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 25e54f1a04e7f0ee377715c52235f8a204734325
    branch: with_privates
a: something #  A pa

Alternatively these attributes can be acessed in python as:

In [31]:
from ruamel.yaml.comments import CommentedMap

exp3 = AnExperiment()
exp3.from_yml('/tmp/defaults.yml')
version_dict = exp3.version

def recursive_lines(dic):
    lines = []
    for k, v in dic.items():
        if type(v) is CommentedMap:
            lines += ['  ' + l for l in recursive_lines(v)]
        else:
            lines += [f'{k}: {v}']
    return lines

print('\n'.join(recursive_lines(version_dict)))

module: /Users/robweston/xmen/examples/<ipython-input-15-64aecc4b7aea>
class: AnExperiment
  local: /Users/robweston/xmen
  remote: https://github.com/robw4/xmen.git
  commit: 25e54f1a04e7f0ee377715c52235f8a204734325
  branch: with_privates


## Command Line Interface
In addition to the python api, each experiment is also able to expose a command line interface with minimal overhead. This is done by adding the follwoing lines to the module the experiment is defined in:

```python
# Import the experiment_parser
from xmen.experiment import Experiment, experiment_parser

class AnExperiment(Experiment):
    # ... Define an experiment

# Add the following lines to the bottom of the module
if __name__ == '__main__':
    args = experiment_parser.parse_args()
    exp = AnExperiment()
    exp.main(args)
```

The `AnExperiment` defined earlier is defined in `experiment.py` module with the above modifications. This allows an `AnExperiment` to be used to generate `default.yml`, to be registered and to be executed all from the command line:

In [32]:
%%bash
python experiment.py -h

usage: experiment.py [-h] [--update YAML_STRING] [--execute PARAMS]
                     [--to_root DIR] [--to_defaults DIR]
                     [--register ROOT NAME]

Run the Experiment command line interface

optional arguments:
  -h, --help            show this help message and exit
  --update YAML_STRING  Update the parameters given by a yaml string. Note
                        this will be called beforeother flags and can be used
                        in combination with --to_root, --to_defaults,and
                        --register.
  --execute PARAMS      Execute the experiment from the given params.yml file.
                        Cannot be called with update.
  --to_root DIR         Generate a run script and defaults.yml file for
                        interfacing with the experiment manager. If the
                        directory does not exist then it is first created.
  --to_defaults DIR     Generate a defaults.yml file from the experiment
                        

For example:

In [33]:
%%bash

# Generate a defaults.yml
python experiment.py --update '{a: updated}' --to_defaults /tmp/command_line/updated_defaults

# Visualise results
echo ""
echo "Generate a defaults.yml file first updating some parameters with a yaml string..."
echo "/tmp/command_line/defaults.yml (a has been updated)"
echo "----------------------------------------------------"
echo "$(cat /tmp/command_line/defaults.yml)"

Updating parameters {'a': 'updated'}

Generate a defaults.yml file first updating some parameters with a yaml string...
/tmp/command_line/defaults.yml (a has been updated)
----------------------------------------------------
created: 03:42PM September 18, 2019  #  The date the last time the parameters of the model were updated.
version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/projects/rad2sim2rad/docs/experiment.py
  class: AnExperiment
  git:
    local: /Users/robweston/projects/rad2sim2rad
    remote: ssh://git@mrgbuild.robots.ox.ac.uk:7999/~robw/rad2sim2rad.git
    commit: 70775458a7599e70540e7f21a408435950320fac
    branch: master
a: updated #  A parameter (default='h')
b: 17 #  Another parameter (default=17)


In [34]:
%%bash
# Registering an experiment
rm -f /tmp/command_line/an_experiment/params.yml

python experiment.py --register /tmp/command_line an_experiment
echo ""
echo "/tmp/command_line/an_experiment/params.yml"
echo "----------------------------------------------------"
echo "$(cat /tmp/command_line/an_experiment/params.yml)"


/tmp/command_line/an_experiment/params.yml
----------------------------------------------------
_root: /tmp/command_line  #  The root directory to which the experiment belongs (should not be set)
_name: an_experiment #  The name of the experiment (should not be set)
_status: registered #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
_created: 11:59AM September 20, 2019 #  The date the last time the parameters of the model were updated.
_purpose: '' #  The purpose for the experiment (should not be set)
_messages: {} #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
_version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/experiment.py
  class: AnExperiment
  git:
    local: /Users/robweston/xmen
    remote: https://github.com/robw4/xmen.git
    commit: 25e54f1a04e7f0ee377715c52235f8a204734325
    branc

In [35]:
%%bash
# Execute an experiment
python experiment.py --execute /tmp/command_line/an_experiment/params.yml

echo ""
echo "After executing the experiment..."
echo "/tmp/command_line/an_experiment/params.yml"
echo "----------------------------------------------------"
echo "$(cat /tmp/command_line/an_experiment/params.yml)"

The experiment state inside run is running for experiment an_experiment, a = h, b = 17

After executing the experiment...
/tmp/command_line/an_experiment/params.yml
----------------------------------------------------
_root: /tmp/command_line  #  The root directory to which the experiment belongs (should not be set)
_name: an_experiment #  The name of the experiment (should not be set)
_status: finished #  The status of the experiment (one of ['default' | 'created' | 'running' | 'error' | 'finished'])
_created: 11:59AM September 20, 2019 #  The date the last time the parameters of the model were updated.
_purpose: '' #  The purpose for the experiment (should not be set)
_messages: #  A dictionary of messages which are able to vary throughout the experiment (should not be set)
  Hi: I am running!
_version: #  A dictonary containing the experiment version information. See `get_version` for more info
  module: /Users/robweston/xmen/examples/experiment.py
  class: AnExperiment
  git:
    l

## The Experiment Manager
The experiment manager is a python command line tool for managing a set of experiments which all share execution code (a `script.sh`) which takes as input parameters (`defaults.yml`). By changing the parameters, each experiment is uniquely defined. Firstly, lets create a root experiment repository from the experiment defined earlier

In [1]:
!python experiment.py --to_root /tmp/experiment_manager/root

#!/bin/bash
# File generated on the 12:12PM September 20, 2019
# GIT:
# - repo /Users/robweston/xmen
# - branch with_privates
# - remote https://github.com/robw4/xmen.git
# - commit 25e54f1a04e7f0ee377715c52235f8a204734325

export PYTHONPATH="${PYTHONPATH}:/Users/robweston/xmen/examples"
python /Users/robweston/xmen/examples/experiment.py --execute ${1}


The above gives a bash script automatically generated in order to run an experiment saved as `script.sh` in the root experiment repository along with a `defaults.yml` file:

In [2]:
!ls -l /tmp/experiment_manager/root

total 24
drwxr-xr-x  2 robweston  wheel   68 20 Sep 12:08 a:first__b:4
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:10 a:first__b:4_1
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:10 a:first__b:4_2
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:10 a:first__b:4_3
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:11 a:first__b:4_4
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:11 a:first__b:4_5
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:10 a:first__b:5
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:10 a:second__b:4
drwxr-xr-x  3 robweston  wheel  102 20 Sep 12:10 a:second__b:5
-rw-r--r--  1 robweston  wheel  526 20 Sep 12:12 defaults.yml
-rw-r--r--  1 robweston  wheel  791 20 Sep 12:11 experiment.yml
-rw-r--r--  1 robweston  wheel  355 20 Sep 12:12 script.sh


For ease of use we first create an alias to the experiment manager. You are encouraged to do this in your `~/.bashrc` or equivalent. Unfortunately ipython does not register previously defined aliases so we need to redefine it here. We also move to the root repository:

In [3]:
alias xmen ~/xmen/python/xmen/experiment_manager.py

First we navigate to the root directory:

In [4]:
%cd /tmp/experiment_manager/root
!rm -f experiment.yml

/private/tmp/experiment_manager/root


Next we initialise the experiment manager

In [5]:
%xmen init

Initilialising the experiment parameter in the root directory produces a `experiment.yml` file. The `exp list` command provides a helpfull output to visualise the state of the current experiements:

In [6]:
%xmen list

No experiments currently registered!

Defaults: 
- created: 12:12PM September 20, 2019
- module: /Users/robweston/xmen/examples/experiment.py
- class: AnExperiment
- git:
   local: /Users/robweston/xmen
   commit: 25e54f1a04e7f0ee377715c52235f8a204734325
   remote: https://github.com/robw4/xmen.git


Multiple experiments are registered with the experiment manager in a single call through the `register` command:

In [ ]:
xmen register '{a: first | second, b: 4 | 5}' 'intial_test'

The special charcter `|` is interpreted as an or operator. Experiments are registered for every possible combination of parameters appearing with `|`. For example in the above 4 experiments have been registered:

In [8]:
%xmen list

        overrides      purpose                     created      status messages                                    commit
0  a:first__b:4_1  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
1    a:first__b:5  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
2   a:second__b:4  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
3   a:second__b:5  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325

Defaults: 
- created: 12:10PM September 20, 2019
- module: /Users/robweston/xmen/examples/experiment.py
- class: AnExperiment
- git:
   local: /Users/robweston/xmen
   commit: 25e54f1a04e7f0ee377715c52235f8a204734325
   remote: https://github.com/robw4/xmen.git


You are also allowed to register multiple experiments with the same overrides. In this case there name is appended by an index `_1` `_2` etc:

In [10]:
%xmen register '{a: first, b: 4}' 'intial_test'
%xmen register '{a: first, b: 4}' 'intial_test'
%xmen list

        overrides      purpose                     created      status messages                                    commit
0  a:first__b:4_1  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
1    a:first__b:5  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
2   a:second__b:4  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
3   a:second__b:5  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
4  a:first__b:4_2  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
5  a:first__b:4_3  intial_test  12:10PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
6  a:first__b:4_4  intial_test  12:11PM September 20, 2019  registered       {}  25e54f1a04e7f0ee377715c52235f8a204734325
7  a:first__b:4_5  intia

Experiments can be run either individually or all together:

In [ ]:
%xmen run 'a:second__b:4' sh

In [32]:
%xmen list

        overrides      purpose                     created      status                 messages                                    commit
0    a:first__b:4  intial_test  11:05AM September 20, 2019  registered                       {}  a692bc7ea31af51890c828ddc836456b1f7776e3
1    a:first__b:5  intial_test  11:05AM September 20, 2019  registered                       {}  a692bc7ea31af51890c828ddc836456b1f7776e3
2   a:second__b:4  intial_test  11:05AM September 20, 2019    finished  {'Hi': 'I am running!'}  a692bc7ea31af51890c828ddc836456b1f7776e3
3   a:second__b:5  intial_test  11:05AM September 20, 2019  registered                       {}  a692bc7ea31af51890c828ddc836456b1f7776e3
4  a:first__b:4_1  intial_test  11:05AM September 20, 2019  registered                       {}  a692bc7ea31af51890c828ddc836456b1f7776e3
5  a:first__b:4_2  intial_test  11:05AM September 20, 2019  registered                       {}  a692bc7ea31af51890c828ddc836456b1f7776e3

Defaults: 
- created 11:05AM Sept

In [33]:
%xmen run '*' sh
%xmen list


Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:4/params.yml
The experiment state inside run is running for experiment a:first__b:4, a = first, b = 4

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:4_1/params.yml
The experiment state inside run is running for experiment a:first__b:4_1, a = first, b = 4

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:4_2/params.yml
The experiment state inside run is running for experiment a:first__b:4_2, a = first, b = 4

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:first__b:5/params.yml
The experiment state inside run is running for experiment a:first__b:5, a = first, b = 5

Running: sh /private/tmp/experiment_manager/root/script.sh /private/tmp/experiment_manager/root/a:second__b:5/params.yml
The experiment state inside ru

Lets now clean up the experiment repository:

In [34]:
# First we unlink all the experiments
%xmen unlink '*'

Removing Experiments...
/private/tmp/experiment_manager/root/a:first__b:4
/private/tmp/experiment_manager/root/a:first__b:4_1
/private/tmp/experiment_manager/root/a:first__b:4_2
/private/tmp/experiment_manager/root/a:first__b:5
/private/tmp/experiment_manager/root/a:second__b:4
/private/tmp/experiment_manager/root/a:second__b:5
Note models are removed from the experiment list only. To remove the model directories runexperiment rm


In [35]:
#Now we remove the experiment folders 
#WARNING: THIS WILL REMOVE ALL EXPERIMENTS THAT ARE NO LONGER LINKED TO THE EXPERIMENT MANAGER
%xmen rm '*'
!echo ""
%xmen list

/private/tmp/experiment_manager/root/a:first__b:4
/private/tmp/experiment_manager/root/a:first__b:4_1
/private/tmp/experiment_manager/root/a:first__b:4_2
/private/tmp/experiment_manager/root/a:first__b:5
/private/tmp/experiment_manager/root/a:second__b:4
/private/tmp/experiment_manager/root/a:second__b:5

No experiments currently registered!

Defaults: 
- created 11:05AM September 20, 2019
- module /Users/robweston/xmen/examples/experiment.py
- class AnExperiment
- git:
   local: /Users/robweston/xmen
   commit: a692bc7ea31af51890c828ddc836456b1f7776e3
   remote: https://github.com/robw4/xmen.git


## The TypedMeta metaclass
The `TypedMeta` meta class stremalines the definition of objects whith many attirbutes that most of the time will simply just take default values. For example consider the following class definition:

In [35]:
class A(object):
    def __init__(self, a=3, b=4, c=3, d=4, e=5, f=6, g=7):
        """An object with some attributes:
        
        Parameters:
            a (int): The first attribute
            b (int): The second attribute
            c (int): The third attribute 
            d (int): The fourth attribute
            e (int): The fith attribute
            g (int): The sixth attribute
        """
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.e = e
        self.f = f
        self.g = g

For a large number of parameters defining parameters in the args, docstrings and attributes quickly becomes tiresome. We could instead think about inferring attributes directly from the arguments of a function. Maybe by updating `self.__dict__` directly after using `argparse` to get the parameters from the function definition or simply only passing `**kwargs` to `__init__`. This seems quite convoluted and will break code completion for many interpreters.

This is where the `TypedMeta` metaclass comes in. Instead we define objects like:

In [68]:
from xmen.utils import TypedMeta
class AWithMeta(object, metaclass=TypedMeta):
    def __init__(self):
        """An object with some attributes"""
        self.a: int = 1   # The first attribute
        self.b: int = 2   # The second attribute
        self.c: int = 3   # The third attribute 
        self.d: int = 4   # The fourth attribute
        self.e: int = 5   # The fith attribute
        self.f: int = 6   # The sixth attribute
        self.g: int = 7   # The seventh attribute

Here we have made use of typed attributes (introduced in python >= 3). In order to allow the defualt values to be overidden we could also consider passing an option keywords arg `**kwargs` and updating the attribute dictionary directly (updating only on keys correpsonding to valid attributes). The `TypedMeta` metaclass automatically adds attributes to the docstring of `AWithMeta` and `AWithMeta.__init__`:

In [69]:
# The doc string of an experiment object is updated
print(AWithMeta.__init__.__doc__)

An object with some attributes

Parameters:
    a (int): The first attribute (default=1)
    b (int): The second attribute (default=2)
    c (int): The third attribute (default=3)
    d (int): The fourth attribute (default=4)
    e (int): The fith attribute (default=5)
    f (int): The sixth attribute (default=6)
    g (int): The seventh attribute (default=7)


The typed Meta class works with both typed and non-typed attribute definitions with or without comments. It will do the best it can with what it has. For example:

In [70]:
class BWithMeta(object, metaclass=TypedMeta):
    def __init__(self):
        """An object with some attributes"""
        self.a: float = 1   # The first attribute
        self.b: int = 2
        self.c = 3   # The third attribute 
        self.d = 4

print(BWithMeta.__doc__)



Parameters:
    a (float): The first attribute (default=1)
    b (int): (default= 2)
    c : The third attribute (default=3)
    d : (default= 4)


> The TypedMeta class is currently only compatible with comments defined on a single line. You are recommended to specify longer comments in the doc-string directly